In [3]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [4]:
env = GridworldEnv()

In [15]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    
    R_s = np.zeros(env.nS)
    P_ss = np.zeros((env.nS, env.nS))
    for s in range(env.nS):
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[s][a]:
                R_s[s] += policy[s,a]*reward*prob
                P_ss[s, next_state] += policy[s,a] * prob
    
    while True:
        V1 = R_s + discount_factor*np.dot(P_ss, V)
        epsilon = np.max(np.abs(V1 - V))
        if epsilon < theta:
            break
        else:
            V = V1
    return np.array(V)

In [17]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [18]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)